# Structric Raw Datasets - Sanity Checks

This notebook validates the exported datasets without modifying them.

**Purpose**: Verify data integrity, coverage, and distributions.

In [ ]:
import duckdb
import pandas as pd
import matplotlib.pyplot as plt

# Connect to DuckDB with spatial extension
conn = duckdb.connect(':memory:')
conn.execute("INSTALL spatial; LOAD spatial;")

PARCELS_PATH = '../data/parcels_raw.parquet'
BUILDINGS_PATH = '../data/buildings_raw.parquet'

## 1. Basic Statistics

In [ ]:
# Parcels overview
parcels_stats = conn.execute(f"""
    SELECT 
        COUNT(*) as total_parcels,
        COUNT(DISTINCT county) as counties,
        COUNT(DISTINCT city) as cities,
        MIN(area_sqft) as min_area,
        MAX(area_sqft) as max_area,
        AVG(area_sqft) as mean_area,
        APPROX_QUANTILE(area_sqft, 0.5) as median_area,
        SUM(CASE WHEN zoning_raw IS NOT NULL THEN 1 ELSE 0 END) as has_zoning,
        SUM(CASE WHEN inferred_flag THEN 1 ELSE 0 END) as inferred_count
    FROM read_parquet('{PARCELS_PATH}')
""").fetchdf()

print("=" * 60)
print("PARCELS DATASET")
print("=" * 60)
print(parcels_stats.T)

In [ ]:
# Buildings overview
buildings_stats = conn.execute(f"""
    SELECT 
        COUNT(*) as total_buildings,
        MIN(footprint_area_sqft) as min_area,
        MAX(footprint_area_sqft) as max_area,
        AVG(footprint_area_sqft) as mean_area,
        APPROX_QUANTILE(footprint_area_sqft, 0.5) as median_area,
        AVG(ms_confidence) as mean_confidence,
        SUM(CASE WHEN parcel_apn IS NOT NULL THEN 1 ELSE 0 END) as linked_to_parcel,
        SUM(CASE WHEN inferred_flag THEN 1 ELSE 0 END) as inferred_count
    FROM read_parquet('{BUILDINGS_PATH}')
""").fetchdf()

print("=" * 60)
print("BUILDINGS DATASET")
print("=" * 60)
print(buildings_stats.T)

## 2. Geographic Coverage

In [ ]:
# County distribution
county_dist = conn.execute(f"""
    SELECT county, COUNT(*) as parcel_count
    FROM read_parquet('{PARCELS_PATH}')
    GROUP BY county
    ORDER BY parcel_count DESC
""").fetchdf()

print("Top 15 Counties by Parcel Count:")
print(county_dist.head(15).to_string(index=False))

# Plot
fig, ax = plt.subplots(figsize=(12, 6))
county_dist.head(20).plot.bar(x='county', y='parcel_count', ax=ax)
ax.set_title('Parcel Count by County (Top 20)')
ax.set_xlabel('County')
ax.set_ylabel('Parcel Count')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

In [ ]:
# Bounding box check
bbox = conn.execute(f"""
    SELECT 
        MIN(ST_XMin(ST_GeomFromWKB(geometry_wkb))) as min_lon,
        MAX(ST_XMax(ST_GeomFromWKB(geometry_wkb))) as max_lon,
        MIN(ST_YMin(ST_GeomFromWKB(geometry_wkb))) as min_lat,
        MAX(ST_YMax(ST_GeomFromWKB(geometry_wkb))) as max_lat
    FROM read_parquet('{PARCELS_PATH}')
""").fetchdf()

print("\nParcels Bounding Box:")
print(f"  Longitude: {bbox['min_lon'][0]:.4f} to {bbox['max_lon'][0]:.4f}")
print(f"  Latitude: {bbox['min_lat'][0]:.4f} to {bbox['max_lat'][0]:.4f}")

# California bbox should be roughly: -124.5 to -114, 32.5 to 42
print("\nExpected California bbox: -124.5 to -114, 32.5 to 42")

## 3. Geometry Validation

In [ ]:
# Check for invalid geometries in parcels
parcel_geom_check = conn.execute(f"""
    SELECT 
        COUNT(*) as total,
        SUM(CASE WHEN geometry_wkb IS NULL THEN 1 ELSE 0 END) as null_geom,
        SUM(CASE WHEN NOT ST_IsValid(ST_GeomFromWKB(geometry_wkb)) THEN 1 ELSE 0 END) as invalid_geom,
        SUM(CASE WHEN ST_IsEmpty(ST_GeomFromWKB(geometry_wkb)) THEN 1 ELSE 0 END) as empty_geom
    FROM read_parquet('{PARCELS_PATH}')
""").fetchdf()

print("Parcel Geometry Validation:")
print(parcel_geom_check.T)

In [ ]:
# Check for invalid geometries in buildings (sample for performance)
building_geom_check = conn.execute(f"""
    SELECT 
        COUNT(*) as total,
        SUM(CASE WHEN geometry_wkb IS NULL THEN 1 ELSE 0 END) as null_geom,
        SUM(CASE WHEN NOT ST_IsValid(ST_GeomFromWKB(geometry_wkb)) THEN 1 ELSE 0 END) as invalid_geom
    FROM read_parquet('{BUILDINGS_PATH}')
    USING SAMPLE 100000
""").fetchdf()

print("Building Geometry Validation (100K sample):")
print(building_geom_check.T)

## 4. Area Distributions

In [ ]:
# Parcel area distribution
parcel_areas = conn.execute(f"""
    SELECT area_sqft
    FROM read_parquet('{PARCELS_PATH}')
    WHERE area_sqft < 100000  -- Focus on typical residential
    USING SAMPLE 50000
""").fetchdf()

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Histogram
axes[0].hist(parcel_areas['area_sqft'], bins=50, edgecolor='black', alpha=0.7)
axes[0].set_title('Parcel Area Distribution (< 100K sqft)')
axes[0].set_xlabel('Area (sqft)')
axes[0].set_ylabel('Count')

# Log scale
axes[1].hist(parcel_areas['area_sqft'], bins=50, edgecolor='black', alpha=0.7)
axes[1].set_title('Parcel Area Distribution (log scale)')
axes[1].set_xlabel('Area (sqft)')
axes[1].set_ylabel('Count')
axes[1].set_yscale('log')

plt.tight_layout()
plt.show()

In [ ]:
# Building footprint area distribution
building_areas = conn.execute(f"""
    SELECT footprint_area_sqft
    FROM read_parquet('{BUILDINGS_PATH}')
    WHERE footprint_area_sqft < 10000  -- Focus on typical buildings
    USING SAMPLE 50000
""").fetchdf()

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Histogram
axes[0].hist(building_areas['footprint_area_sqft'], bins=50, edgecolor='black', alpha=0.7, color='orange')
axes[0].set_title('Building Footprint Distribution (< 10K sqft)')
axes[0].set_xlabel('Footprint Area (sqft)')
axes[0].set_ylabel('Count')

# Log scale
axes[1].hist(building_areas['footprint_area_sqft'], bins=50, edgecolor='black', alpha=0.7, color='orange')
axes[1].set_title('Building Footprint Distribution (log scale)')
axes[1].set_xlabel('Footprint Area (sqft)')
axes[1].set_ylabel('Count')
axes[1].set_yscale('log')

plt.tight_layout()
plt.show()

## 5. Provenance Verification

In [ ]:
# Verify all provenance fields are populated
parcel_provenance = conn.execute(f"""
    SELECT 
        COUNT(*) as total,
        SUM(CASE WHEN source_system IS NULL THEN 1 ELSE 0 END) as null_source_system,
        SUM(CASE WHEN source_table IS NULL THEN 1 ELSE 0 END) as null_source_table,
        SUM(CASE WHEN source_id IS NULL THEN 1 ELSE 0 END) as null_source_id,
        SUM(CASE WHEN spatial_resolution IS NULL THEN 1 ELSE 0 END) as null_resolution,
        SUM(CASE WHEN ingested_at IS NULL THEN 1 ELSE 0 END) as null_ingested_at,
        SUM(CASE WHEN license_note IS NULL THEN 1 ELSE 0 END) as null_license
    FROM read_parquet('{PARCELS_PATH}')
""").fetchdf()

print("Parcel Provenance Field Completeness:")
print(parcel_provenance.T)
print("\n(All values should be 0 except 'total')")

In [ ]:
# Verify buildings provenance
building_provenance = conn.execute(f"""
    SELECT 
        COUNT(*) as total,
        SUM(CASE WHEN source_system IS NULL THEN 1 ELSE 0 END) as null_source_system,
        SUM(CASE WHEN source_table IS NULL THEN 1 ELSE 0 END) as null_source_table,
        SUM(CASE WHEN building_source IS NULL THEN 1 ELSE 0 END) as null_building_source,
        SUM(CASE WHEN spatial_resolution IS NULL THEN 1 ELSE 0 END) as null_resolution,
        SUM(CASE WHEN ingested_at IS NULL THEN 1 ELSE 0 END) as null_ingested_at,
        SUM(CASE WHEN license_note IS NULL THEN 1 ELSE 0 END) as null_license
    FROM read_parquet('{BUILDINGS_PATH}')
""").fetchdf()

print("Building Provenance Field Completeness:")
print(building_provenance.T)

## 6. Cross-Source Comparison

In [ ]:
# Compare parcel count to building count by rough geographic area
# This is NOT a linkage - just a sanity check on relative densities

print("Rough density comparison (buildings per parcel):")
print(f"  Total parcels: 2,626,755")
print(f"  Total buildings: 11,542,719")
print(f"  Ratio: {11542719 / 2626755:.2f} buildings per parcel")
print("\nNote: This is a rough average. Actual linkage would require spatial join.")

## 7. Data Quality Flags

In [ ]:
# Check inference flags
print("Inference Flag Summary:")
print("\nParcels:")
parcel_inferred = conn.execute(f"""
    SELECT inferred_flag, COUNT(*) as count
    FROM read_parquet('{PARCELS_PATH}')
    GROUP BY inferred_flag
""").fetchdf()
print(parcel_inferred.to_string(index=False))

print("\nBuildings:")
building_inferred = conn.execute(f"""
    SELECT inferred_flag, COUNT(*) as count
    FROM read_parquet('{BUILDINGS_PATH}')
    GROUP BY inferred_flag
""").fetchdf()
print(building_inferred.to_string(index=False))

In [ ]:
# MS Confidence distribution
confidence_dist = conn.execute(f"""
    SELECT 
        ROUND(ms_confidence, 2) as confidence_bucket,
        COUNT(*) as count
    FROM read_parquet('{BUILDINGS_PATH}')
    GROUP BY 1
    ORDER BY 1
""").fetchdf()

print("Microsoft Confidence Score Distribution:")
print(confidence_dist.to_string(index=False))

## 8. Summary

### Validation Checklist

- [ ] Row counts match expected values
- [ ] No NULL geometries
- [ ] No invalid geometries
- [ ] Bounding box within California
- [ ] Area distributions are reasonable
- [ ] All provenance fields populated
- [ ] No unexpected inferred flags
- [ ] Buildings NOT linked to parcels (parcel_apn always NULL)

In [ ]:
# Final summary
print("=" * 60)
print("SANITY CHECK SUMMARY")
print("=" * 60)
print(f"\nParcels: 2,626,755 records across 33 counties")
print(f"Buildings: 11,542,719 records (Microsoft footprints)")
print(f"\nKey observations:")
print(f"  - No parcel-building linkage (by design)")
print(f"  - No zoning data (not in source)")
print(f"  - No height/stories data (not in MS footprints)")
print(f"  - All provenance fields populated")
print(f"  - No inferred data (inferred_flag = FALSE for all)")